In [1]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm.notebook import tqdm

import itertools
from collections import defaultdict
from helper_functions import get_picture_filenames_from_folder, load_and_transform_pictures, display_blur_sharp_and_pred_images, calc_mae

In [2]:
# Setup
plt.rcParams["figure.figsize"] = [9, 16]

In [3]:
# Path helpers
blurred_train_path = os.path.join('clean_data', 'train', 'blur')
sharp_train_path = os.path.join('clean_data', 'train', 'sharp')
blurred_test_path = os.path.join('clean_data', 'test', 'blur')
sharp_test_path = os.path.join('clean_data', 'test', 'sharp')

In [4]:
# The fourier transform is based on https://towardsdatascience.com/how-to-de-blur-images-without-training-neural-networks-72f8597c0014
# Best results were looked for with changing the wiener factor constant

def gaussian_filter(kernel_size,img,sigma=1, muu=0):
    x, y = np.meshgrid(np.linspace(-1, 1, kernel_size),
                       np.linspace(-1, 1, kernel_size))
    dst = np.sqrt(x**2+y**2)
    normal = 1/(((2*np.pi)**0.5)*sigma)
    gauss = np.exp(-((dst-muu)**2 / (2.0 * sigma**2))) * normal
    gauss = np.pad(gauss, [(0, img.shape[0] - gauss.shape[0]), (0, img.shape[1] - gauss.shape[1])], 'constant')
    return gauss

def fft_deblur(img,kernel_size,kernel_sigma=5,factor='wiener',const=0.002):
    gauss = gaussian_filter(kernel_size,img,kernel_sigma)
    img_fft = np.fft.fft2(img)
    gauss_fft = np.fft.fft2(gauss)
    weiner_factor = 1 / (1+(const/np.abs(gauss_fft)**2))
    if factor!='wiener':
        weiner_factor = factor
    recon = img_fft/gauss_fft
    recon*=weiner_factor
    recon = np.abs(np.fft.ifft2(recon))
    return recon

In [5]:
# Fourier transform on coloured data
def fft_deblur_color(img,kernel_size,kernel_sigma=5,factor='wiener',const=0.002):
    new_img = np.zeros(img.shape)
    for dim in range(3):
        new_img[:,:,dim] = fft_deblur(img[:,:,dim], kernel_size = kernel_size, const = const)
    new_img = np.clip(new_img, 0, 1).astype('float64')
    return new_img

In [6]:
# Finding best hyperparameters using train data
filenames = get_picture_filenames_from_folder(blurred_train_path)

const_list = [0.001, 0.01, 0.1, 1, 10, 20]
kernel_size_list = [1, 3, 5, 7, 9, 11, 13, 15]
kernel_sigma_list = [1, 5, 10, 15, 20]


losses = defaultdict(list)
loss_fn = calc_mae
for i, filename in enumerate(filenames[:10]):
    print(i)
    blurred = cv2.imread(os.path.join(blurred_train_path, filename))/255
    sharp = cv2.imread(os.path.join(sharp_train_path, filename))/255
    
    for const, kernel_size, kernel_sigma in itertools.product(
        const_list, kernel_size_list, kernel_sigma_list):
        deblurred = fft_deblur_color(blurred, kernel_size, kernel_sigma = kernel_sigma, const = const)
        losses[(const, kernel_size, kernel_sigma)].append(calc_mae(deblurred, sharp))

0
1
2
3
4
5
6
7
8
9


In [7]:
lowest_loss = 100
lowest_loss_params = (0, 0, 0)

for key, loss_list in losses.items():
    mean_loss = np.mean(loss_list)
    print(key, np.mean(loss_list))
    if mean_loss < lowest_loss:
        lowest_loss = mean_loss
        lowest_loss_params = key

(0.001, 1, 1) 0.5413799950922815
(0.001, 1, 5) 0.5413799950922815
(0.001, 1, 10) 0.5413799950922815
(0.001, 1, 15) 0.5413799950922815
(0.001, 1, 20) 0.5413799950922815
(0.001, 3, 1) 0.16507036575833794
(0.001, 3, 5) 0.16507036575833794
(0.001, 3, 10) 0.16507036575833794
(0.001, 3, 15) 0.16507036575833794
(0.001, 3, 20) 0.16507036575833794
(0.001, 5, 1) 0.22067626709174912
(0.001, 5, 5) 0.22067626709174912
(0.001, 5, 10) 0.22067626709174912
(0.001, 5, 15) 0.22067626709174912
(0.001, 5, 20) 0.22067626709174912
(0.001, 7, 1) 0.31163620877530246
(0.001, 7, 5) 0.31163620877530246
(0.001, 7, 10) 0.31163620877530246
(0.001, 7, 15) 0.31163620877530246
(0.001, 7, 20) 0.31163620877530246
(0.001, 9, 1) 0.3401192904444188
(0.001, 9, 5) 0.3401192904444188
(0.001, 9, 10) 0.3401192904444188
(0.001, 9, 15) 0.3401192904444188
(0.001, 9, 20) 0.3401192904444188
(0.001, 11, 1) 0.3509264468827138
(0.001, 11, 5) 0.3509264468827138
(0.001, 11, 10) 0.3509264468827138
(0.001, 11, 15) 0.3509264468827138
(0.001,

In [8]:
print(lowest_loss)

0.08429371835389096


In [9]:
print(lowest_loss_params)

(0.1, 3, 1)


In [10]:
# Best found params: 0.1, 3, 1

# Use best hyperparameters
filenames = get_picture_filenames_from_folder(blurred_test_path)

const_ = 0.1
kernel_size_ = 3
kernel_sigma_ = 1

losses = []

for i, filename in enumerate(filenames):
    blurred = cv2.imread(os.path.join(blurred_test_path, filename))/255
    sharp = cv2.imread(os.path.join(sharp_test_path, filename))/255
    deblurred = fft_deblur_color(blurred, kernel_size = kernel_size_, kernel_sigma = kernel_sigma_, const = const_)
    losses.append(calc_mae(deblurred, sharp))
print("mean loss:", np.mean(losses))

mean loss: 0.0876350489954407
